In [ ]:
import requests

class Node:
    def __init__(self, id, x, y):
        self.id = id
        self.x = x
        self.y = y


class Edge:
    def __init__(self, node1, node2, weight):
        self.node1 = node1
        self.node2 = node2
        self.weight = weight


class Graph:
    def __init__(self):
        self.nodes = []
        self.edges = []

    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, edge):
        self.edges.append(edge)

    def get_node_by_id(self, id):
        for node in self.nodes:
            if node.id == id:
                return node


class DatasetExtractor():

    def __init__(self):
        self.graph = Graph()

    def api_call_to_get_edge_weight(self, from_id, to_id):
        node1 = self.graph.get_node_by_id(from_id)
        node2 = self.graph.get_node_by_id(to_id)

        # Calculate distance between two nodes
        long1 = float(node1.x) / 1000000
        lat1 = float(node1.y) / 1000000

        long2 = float(node2.x) / 1000000
        lat2 = float(node2.y) / 1000000

        # Make the API call
        url = f"http://router.project-osrm.org/route/v1/driving/{long1},{lat1};{long2},{lat2}"
        response = requests.get(url)

        if response.status_code == 200:
            result = response.json()

            # Extract distance and duration from the response
            distance = result['routes'][0]['distance']
            duration = result['routes'][0]['duration']

            print(f"From {from_id} to {to_id} - distance: {distance} - duration: {duration}")

            return distance, duration
        else:
            print(f"Error: Unable to fetch data from API. Status code {response.status_code}")
            return None, None


    def extract_dataset(self, size, write_to_file):
        content = ""

        # Read from file USA-road-d.USA.co and add nodes
        filename = "../data/USA-road-d.USA.co"
        with open(filename, 'r') as file:
            for line in file:
                if line.startswith('v'):
                    tokens = line.split()
                    id = int(tokens[1])
                    x = int(tokens[2])
                    y = int(tokens[3])
                    if id > size:
                        continue
                    if write_to_file:
                        content += f"{id} {x} {y}\n"
                    node = Node(id, x, y)
                    self.graph.add_node(node)

        print(f"Number of nodes: {len(self.graph.nodes)}")
        # Write to file
        if write_to_file:
            filename = "../data/nodes.txt"
            with open(filename, 'w') as file:
                file.write(content)


        # Read from file USA-road-d.USA.gr and add edges based on distance
        filename = "../data/USA-road-d.USA.gr"
        with open(filename, 'r') as file:
            for line in file:
                if line.startswith('a'):
                    tokens = line.split()
                    from_id = int(tokens[1])
                    to_id = int(tokens[2])
                    

                    if from_id > size or to_id > size:
                        continue

                    distance,duration = self.api_call_to_get_edge_weight(from_id, to_id)
                    weight = (distance , duration)

                    if write_to_file:
                        if write_to_file:
                            # Open the file in append mode and write content
                            with open("../data/edges.txt", 'a') as file:
                                file.write(f"{from_id} {to_id} {distance} {duration}\n")

                    node1 = self.graph.get_node_by_id(from_id)
                    node2 = self.graph.get_node_by_id(to_id)
                    edge = Edge(node1, node2, weight)
                    self.graph.add_edge(edge)

        print(f"Number of edges: {len(self.graph.edges)}")

        return self.graph


if __name__ == "__main__":
    dataset_extractor = DatasetExtractor()
    size = 250000
    graph = dataset_extractor.extract_dataset(size, True)
    print("Graph created successfully!")
